# Train models to predict faiure probability at hourly resolution

## Features :

*Weather*
-
-
-

*Load*
-
-
-

*Chronological*
-
-
-

In [1]:
# Import libraries

# Data processing and manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.cluster import KMeans
import xgboost as xgb

# Other
import importlib
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))
import inferenceModels as im    # your file
import preprocess_data as ppd  



# Mutual information
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif



from typing import Iterable
from pathlib import Path



importlib.reload(im)

<module 'inferenceModels' from '/Users/malo/MIT Dropbox/Malo Lahogue/Research/Generator-outage-probability/src/inferenceModels.py'>

## Load data

## Get features and data source

In [2]:
weather_data_file = "../Data/hourly/hourly_weather_by_state.csv"
power_load_file = "../Data/hourly/hourly_load_by_state.csv"
failure_data_file = "../Data/hourly/hourly_failure_dataset_compressed.csv"

weather_data = pd.read_csv(weather_data_file, parse_dates=["datetime"])
power_load_data = pd.read_csv(power_load_file, parse_dates=["UTC time"])

In [3]:
feature_names = list(weather_data.columns) + list(power_load_data.columns) + ['Season', 'Month', 'DayOfWeek', 'DayOfYear', 'Holiday', 'Weekend', 'Technology']
feature_names = list(set(feature_names)-set(['datetime', 'UTC time', 'Datetime_UTC', 'Datetime']))
feature_names = list(set([(name[0].upper() + name[1:]) if isinstance(name, str) and name else name for name in feature_names]))
print(f"{len(feature_names)} features:", feature_names)
technologies = ['Gas Turbine/Jet Engine (Simple Cycle Operation)']

39 features: ['CDD3d', 'DayOfYear', '24h_max_load', 'Relative_humidity', 'Month', 'Load', 'DayOfWeek', '24h_min_load', 'Extreme_heat', 'HDD3d', 'Pressure_3hr_change', 'HDD', 'State', 'Season', 'Precip_3d', 'Extreme_cold', 'Tmean', 'CDD', 'Tmin', 'Weekend', 'Holiday', 'FDD', 'Precipitation', 'Wind_speed', 'Dew_point_temperature', 'Snow_depth', 'FDD3d', 'Heat_index', 'Sea_level_pressure', 'Wet_bulb_temperature', 'Temperature', '2d_load_sum', 'Precip_1d', 'Wind_chill', 'Hourly_load_change', '1d_load_sum', 'Tmax', 'Station_level_pressure', 'Technology']


## Test and count per technology

In [8]:
importlib.reload(im)
train_val_df, test_df, feature_names, target_columns, integer_encoding = ppd.preprocess_data(failure_data_path=failure_data_file,
                                                                weather_data_path=weather_data_file,
                                                                power_load_data_path=power_load_file,
                                                                feature_names=feature_names,
                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                initial_MC_state_filter='A',
                                                                )

In [4]:
importlib.reload(im)
def count_data(technologies):
    print(f"Counting data for technologies: {technologies}")
    states = ['A', 'D', 'U']
    counts = defaultdict(lambda: defaultdict(int))
    for si in states:
        print(f"  Initial state: {si}")
        init_df , _, _, integer_encoding = im.preprocess_data(failure_data_path=failure_data_file,
                                                                weather_data_path=weather_data_file,
                                                                power_load_data_path=power_load_file,
                                                                feature_names=feature_names,
                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                initial_MC_state_filter=si,
                                                                technology_filter=technologies
                                                                )
        for sf, sf_code in integer_encoding['Final_gen_state'].items():
            count = len(init_df[init_df['Final_gen_state'] == sf_code])
            counts[si][sf] = count
    return pd.DataFrame(counts)

In [22]:
integer_encoding['Technologies']

{'CC GT units ': 0,
 'CC steam units': 1,
 'Co-generator Block ': 2,
 'CoG GT units': 3,
 'CoG steam units ': 4,
 'Combined Cycle Block': 5,
 'Fluidized Bed': 6,
 'Fossil-Steam': 7,
 'Gas Turbine/Jet Engine (Simple Cycle Operation)': 8,
 'Gas Turbine/Jet Engine with HSRG': 9,
 'Geothermal': 10,
 'Internal Combustion/Reciprocating Engines': 11,
 'Miscellaneous': 12,
 'Multi-boiler/Multi-turbine': 13,
 'Nuclear': 14,
 'Pumped Storage/Hydro': 15}

In [5]:
nuc_tech = ['Nuclear']
hydro_tech = ['Pumped Storage/Hydro']
geothermal_tech = ['Geothermal']
thermal_tech = ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine with HSRG', 'Internal Combustion/Reciprocating Engines','Multi-boiler/Multi-turbine']

In [7]:
importlib.reload(im)
count_nuc = count_data(nuc_tech)
print(count_nuc)
print(f"    Total nuclear entries: {count_nuc.values.sum()}\n")
count_hydro = count_data(hydro_tech)
print(count_hydro)
print(f"    Total hydro entries: {count_hydro.values.sum()}\n")
count_geo = count_data(geothermal_tech)
print(count_geo)
print(f"    Total geothermal entries: {count_geo.values.sum()}\n")
count_thermal = count_data(thermal_tech)
print(count_thermal)
print(f"    Total thermal entries: {count_thermal.values.sum()}\n")

Counting data for technologies: ['Nuclear']
  Initial state: A
  Initial state: D
  Initial state: U
         A       D       U
A  2181184    3012     311
D     2928  633172     264
U      370     199  108113
    Total nuclear entries: 2929553

Counting data for technologies: ['Pumped Storage/Hydro']
  Initial state: A
  Initial state: D
  Initial state: U
         A       D        U
A  2935393    2216    23278
D     2163  823732      207
U    22651     170  1152211
    Total hydro entries: 4962021

Counting data for technologies: ['Geothermal']
  Initial state: A
  Initial state: D
  Initial state: U
        A      D      U
A  210507    670   1043
D     667  40421     35
U     897     33  37660
    Total geothermal entries: 291933

Counting data for technologies: ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine

# Training

In [4]:
nuc_tech = ['Nuclear']
hydro_tech = ['Pumped Storage/Hydro']
geothermal_tech = ['Geothermal']
thermal_tech = ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine with HSRG', 'Internal Combustion/Reciprocating Engines','Multi-boiler/Multi-turbine']

technologies = thermal_tech

In [5]:
initial_state = 'A'
state = 'New York'
test_periods = [(pd.Timestamp('2022-01-01'), pd.Timestamp('2023-12-31'))]

In [6]:
importlib.reload(im)
importlib.reload(ppd)
train_val_df, test_df, feature_names, target_columns, integer_encoding = ppd.preprocess_data(failure_data_path=failure_data_file,
                                                                                weather_data_path=weather_data_file,
                                                                                power_load_data_path=power_load_file,
                                                                                feature_names=feature_names,
                                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                                initial_MC_state_filter=initial_state,
                                                                                technology_filter=technologies,
                                                                                test_periods=test_periods,
                                                                                state_filter=state
                                                                                )

In [7]:
print(f"Train/Val Dataset shape: {train_val_df.shape}")
print(f"Test Dataset shape: {test_df.shape}")
print(f"Feature names ({len(feature_names)}): {feature_names}")
print(f"Target columns: {target_columns}")

Train/Val Dataset shape: (603243, 56)
Test Dataset shape: (146373, 56)
Feature names (53): ['CDD3d', '24h_max_load', 'Relative_humidity', 'Load', 'Extreme_heat', '24h_min_load', 'HDD3d', 'Pressure_3hr_change', 'HDD', 'Precip_3d', 'Extreme_cold', 'Tmean', 'CDD', 'Tmin', 'Weekend', 'Holiday', 'FDD', 'Precipitation', 'Wind_speed', 'Dew_point_temperature', 'Snow_depth', 'FDD3d', 'Heat_index', 'Sea_level_pressure', 'Wet_bulb_temperature', 'Temperature', '2d_load_sum', 'Precip_1d', 'Wind_chill', 'Hourly_load_change', '1d_load_sum', 'Tmax', 'Station_level_pressure', 'Heat_index_isnan', 'Wind_chill_isnan', 'Technology_CC GT units ', 'Technology_CC steam units', 'Technology_Co-generator Block ', 'Technology_CoG GT units', 'Technology_CoG steam units ', 'Technology_Combined Cycle Block', 'Technology_Fossil-Steam', 'Technology_Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Technology_Gas Turbine/Jet Engine with HSRG', 'Technology_Multi-boiler/Multi-turbine', 'Season_sin', 'Season_cos', 'Month

In [15]:
# Standardize all continuous features (exclude one-hots and raw categorical/cyclic markers)
exclude = {"Holiday", "Weekend", "Season", "Month", "DayOfWeek", "DayOfYear"}
stand_cols = [f for f in feature_names if not f.startswith("State_") and not f.startswith("Technology_") and not f.endswith("_isnan") and not f.endswith("_sin") and not f.endswith("_cos") and f not in exclude]
print(f"Standardized features ({len(stand_cols)}): {stand_cols}")

Standardized features (31): ['CDD3d', '24h_max_load', 'Relative_humidity', 'Load', 'Extreme_heat', '24h_min_load', 'HDD3d', 'Pressure_3hr_change', 'HDD', 'Precip_3d', 'Extreme_cold', 'Tmean', 'CDD', 'Tmin', 'FDD', 'Precipitation', 'Wind_speed', 'Dew_point_temperature', 'Snow_depth', 'FDD3d', 'Heat_index', 'Sea_level_pressure', 'Wet_bulb_temperature', 'Temperature', '2d_load_sum', 'Precip_1d', 'Wind_chill', 'Hourly_load_change', '1d_load_sum', 'Tmax', 'Station_level_pressure']


## XGBoost

In [12]:
importlib.reload(im)

dataset_subset = train_val_df.iloc[:1000000, :].copy()

xgb_model = im.xgboostModel(verbose=True)
xgb_model.build_model(max_depth=6,
                      eta=0.07,
                      gamma=0.8,
                      reg_lambda=1,
                      num_boost_round=100,
                      feature_cols=feature_names,
                      target_cols=target_columns,
                      num_classes=3,
                      eval_metric='mlogloss', # rmse, logloss, mae, mape
                      objective='multi:softprob',
                      subsample=1)

xgb_model.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1,  standardize=False)#, standardize=stand_cols_freq)

xgb_model.train_model(weights_data=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=10,
              enable_categorical=False, eta=0.07, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0.8,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, ...)
[0]	validation_0-mlogloss:0.99659
[1]	validation_0-mlogloss:0.90720
[2]	validation_0-mlogloss:0.82819
[3]	validation_0-mlogloss:0.75786
[4]	validation_0-mlogloss:0.69489
[5]	validation_0-mlogloss:0.63825
[6]	validation_0-mlogloss:0.58711
[7]	validation_

In [38]:
xgb_model.predict(dataset.iloc[:5, :])

array([[0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749]], dtype=float32)

## MLP

In [60]:
importlib.reload(im)

dataset_subset = train_val_df.iloc[:10000, :].copy()

mlp = im.MLP(verbose=True)
mlp.build_model(feature_cols=feature_names, 
                target_cols=target_columns, 
                num_classes=3,
                hidden_sizes=[100, 100, 100, 50], activations=['relu', 'relu', 'relu', 'relu'])

mlp.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1, standardize=stand_cols, reweight_train_data_density='Temperature', reweight_power=2.0)


Sequential(
  (linear_0): Linear(in_features=53, out_features=100, bias=True)
  (activation_0): ReLU()
  (linear_1): Linear(in_features=100, out_features=100, bias=True)
  (activation_1): ReLU()
  (linear_2): Linear(in_features=100, out_features=100, bias=True)
  (activation_2): ReLU()
  (linear_3): Linear(in_features=100, out_features=50, bias=True)
  (activation_3): ReLU()
  (linear_out): Linear(in_features=50, out_features=3, bias=True)
)
Input dim: 53 | Output dim: 3 | Trainable params: 30,803
Reweighting training data by 1/density of column 'Temperature'...


In [61]:
mlp.train_model(optimizer='adam', loss='focal_loss',
                focal_loss_gamma_schedule='exponential',
                focal_loss_alpha=[1,1,1], focal_loss_gamma=0,
                regularization_type='L2', lambda_reg=1e-3,
                weights_data=True,
                epochs=10, batch_size=5, lr=2e-4, device='cpu')

Epoch 001: train=3.7690e-02 | val=1.2363e-02
Epoch 002: train=3.2713e-02 | val=9.9762e-03
Epoch 003: train=3.3297e-02 | val=1.9514e-02
Epoch 004: train=3.9176e-02 | val=1.3500e-02
Epoch 005: train=4.1803e-02 | val=2.4542e-02
Epoch 006: train=4.1536e-02 | val=1.0901e-02
Epoch 007: train=5.6667e-02 | val=1.2647e-02
Epoch 008: train=4.2354e-02 | val=1.2610e-02
Epoch 009: train=3.7869e-02 | val=1.1062e-02
Epoch 010: train=4.7850e-02 | val=1.8880e-02


In [107]:
mlp.val_loss_per_logit

defaultdict(list,
            {0: [7.054250881345062e-05,
              4.2150184537357795e-05,
              0.00011357557498170507,
              6.556489342157142e-05,
              0.0001544277309625373,
              4.561129373431877e-05,
              5.783020603139243e-05,
              5.542622475123543e-05,
              4.110410624769956e-05,
              0.00011463788754309336],
             2: [5.300003638634315,
              6.080329895019531,
              5.269192035381611,
              5.697074890136719,
              4.563824286827674,
              6.487308795635517,
              6.275146484375,
              5.894508948692908,
              6.719149662898137,
              4.932462838979868],
             1: [5.757810865129743,
              6.1578810555594305,
              6.524968828473773,
              6.896753583635602,
              5.92731693812779,
              6.557816096714565,
              6.560573032924107,
              7.336990901402065,
       

In [ ]:
mlp.train_model(optimizer='adam', loss='focal_loss',
                focal_loss_alpha=1, focal_loss_gamma=2,
                regularization_type='L2', lambda_reg=1e-3,
                weights_data=True,
                epochs=10, batch_size=10, lr=2e-4, device='cpu')

# Debug

In [1]:
# Import libraries

# Data processing and manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.cluster import KMeans
import xgboost as xgb

# Other
import importlib
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))
import inferenceModels as im    # your file



# Mutual information
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif



from typing import Iterable
from pathlib import Path



importlib.reload(im)

<module 'inferenceModels' from '/Users/malo/MIT Dropbox/Malo Lahogue/Research/Generator-outage-probability/src/inferenceModels.py'>

In [2]:
weather_data_file = "../Data/hourly/hourly_weather_by_state.csv"
power_load_file = "../Data/hourly/hourly_load_by_state.csv"
failure_data_file = "../Data/hourly/hourly_failure_dataset_compressed.csv"

weather_data = pd.read_csv(weather_data_file, parse_dates=["datetime"])
power_load_data = pd.read_csv(power_load_file, parse_dates=["UTC time"])


feature_names = list(weather_data.columns) + list(power_load_data.columns) + ['Season', 'Month', 'DayOfWeek', 'DayOfYear', 'Holiday', 'Weekend', 'Technology']
feature_names = list(set(feature_names)-set(['datetime', 'UTC time', 'Datetime_UTC', 'Datetime']))
feature_names = list(set([(name[0].upper() + name[1:]) if isinstance(name, str) and name else name for name in feature_names]))
print(f"{len(feature_names)} features:", feature_names)

39 features: ['CDD3d', 'CDD', 'DayOfWeek', 'Heat_index', 'Wind_speed', 'Technology', '24h_max_load', '1d_load_sum', 'FDD3d', 'State', 'Dew_point_temperature', 'Load', 'Extreme_cold', 'Station_level_pressure', 'Tmin', 'Wind_chill', 'Temperature', 'Month', 'HDD', 'Precip_3d', 'Sea_level_pressure', 'Tmean', 'Wet_bulb_temperature', '24h_min_load', 'Pressure_3hr_change', 'Snow_depth', 'Season', '2d_load_sum', 'Tmax', 'Relative_humidity', 'Holiday', 'Precipitation', 'Weekend', 'DayOfYear', 'FDD', 'Hourly_load_change', 'HDD3d', 'Extreme_heat', 'Precip_1d']


In [3]:
dataset_subset = pd.read_csv("../Data/hourly/hourly_failure_dataset_thermal_A_subset.csv")
dataset_subset

,Relative_humidity,State_VIRGINIA,Extreme_cold,State_VERMONT,State_CALIFORNIA,Month_cos,State_OREGON,State_ALABAMA,HDD3d,State_MINNESOTA,...,State_DELAWARE,DayOfYear_cos,Dew_point_temperature,State_IDAHO,State_MASSACHUSETTS,Sea_level_pressure,State_NEW YORK,State_ILLINOIS,Final_gen_state,Data_weight
0,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,2.0
1,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,1.0
2,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,6.0
3,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,8.0
4,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,3.0
996,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,1.0
997,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,21.0
998,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,36.0


In [4]:
target_columns = ['Final_gen_state']
feature_names = ['State_INDIANA', 'State_MISSISSIPPI', 'State_WASHINGTON', 'Pressure_3hr_change', 'Tmax', 'State_CONNECTICUT', 'State_WEST VIRGINIA', 'Season_sin', 'Station_level_pressure', 'Wind_speed', 'HDD3d', 'Dew_point_temperature', 'State_ILLINOIS', 'Season_cos', 'CDD3d', 'State_NEW HAMPSHIRE', 'Temperature', 'Technology', 'FDD3d', 'State_MAINE', 'State_GEORGIA', 'Holiday', 'DayOfYear_cos', 'Extreme_heat', 'DayOfYear_sin', 'Precip_3d', 'Sea_level_pressure', 'State_KANSAS', 'State_OREGON', 'State_MISSOURI', 'State_NEW YORK', 'Relative_humidity', 'State_SOUTH CAROLINA', 'State_OKLAHOMA', 'Hourly_load_change', '1d_load_sum', 'State_NEBRASKA', 'Wind_chill_isnan', 'State_DELAWARE', 'State_OHIO', 'Heat_index', 'State_TEXAS', 'State_ALABAMA', 'State_CALIFORNIA', 'Snow_depth', 'Month_sin', 'State_WYOMING', 'Precip_1d', 'FDD', '2d_load_sum', 'Wet_bulb_temperature', 'DayOfWeek_cos', 'HDD', 'Precipitation', 'State_MARYLAND', '24h_min_load', 'State_NORTH CAROLINA', 'State_ARKANSAS', 'Month_cos', 'State_SOUTH DAKOTA', 'State_MASSACHUSETTS', 'State_FLORIDA', 'State_COLORADO', 'State_UTAH', 'Tmean', 'State_NEVADA', 'Tmin', 'State_NORTH DAKOTA', 'State_LOUISIANA', 'State_TENNESSEE', 'State_MONTANA', 'State_PENNSYLVANIA', 'Load', 'State_VERMONT', 'Extreme_cold', 'Weekend', 'Heat_index_isnan', 'CDD', 'State_MINNESOTA', 'State_NEW JERSEY', 'State_KENTUCKY', 'DayOfWeek_sin', 'State_ARIZONA', '24h_max_load', 'State_IDAHO', 'Wind_chill', 'State_IOWA', 'State_VIRGINIA', 'State_MICHIGAN', 'State_RHODE ISLAND', 'State_NEW MEXICO', 'State_WISCONSIN']

In [5]:
print(f"Dataset shape: {dataset_subset.shape}")
print(f"Feature names ({len(feature_names)}): {feature_names}")
print(f"Target columns: {target_columns}")

Dataset shape: (1000, 94)
Feature names (92): ['State_INDIANA', 'State_MISSISSIPPI', 'State_WASHINGTON', 'Pressure_3hr_change', 'Tmax', 'State_CONNECTICUT', 'State_WEST VIRGINIA', 'Season_sin', 'Station_level_pressure', 'Wind_speed', 'HDD3d', 'Dew_point_temperature', 'State_ILLINOIS', 'Season_cos', 'CDD3d', 'State_NEW HAMPSHIRE', 'Temperature', 'Technology', 'FDD3d', 'State_MAINE', 'State_GEORGIA', 'Holiday', 'DayOfYear_cos', 'Extreme_heat', 'DayOfYear_sin', 'Precip_3d', 'Sea_level_pressure', 'State_KANSAS', 'State_OREGON', 'State_MISSOURI', 'State_NEW YORK', 'Relative_humidity', 'State_SOUTH CAROLINA', 'State_OKLAHOMA', 'Hourly_load_change', '1d_load_sum', 'State_NEBRASKA', 'Wind_chill_isnan', 'State_DELAWARE', 'State_OHIO', 'Heat_index', 'State_TEXAS', 'State_ALABAMA', 'State_CALIFORNIA', 'Snow_depth', 'Month_sin', 'State_WYOMING', 'Precip_1d', 'FDD', '2d_load_sum', 'Wet_bulb_temperature', 'DayOfWeek_cos', 'HDD', 'Precipitation', 'State_MARYLAND', '24h_min_load', 'State_NORTH CAROLIN

In [6]:
assert np.isfinite(dataset_subset.select_dtypes(float)).all().all()

In [7]:
importlib.reload(im)

mlp = im.MLP(verbose=True)
mlp.build_model(feature_cols=feature_names, target_cols=target_columns, num_classes=3,
                hidden_sizes=[100, 100, 100, 50], activations=['relu', 'relu', 'relu', 'relu'])

mlp.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1, test_ratio=0.1)#, standardize=stand_cols_freq)


Sequential(
  (linear_0): Linear(in_features=92, out_features=100, bias=True)
  (activation_0): ReLU()
  (linear_1): Linear(in_features=100, out_features=100, bias=True)
  (activation_1): ReLU()
  (linear_2): Linear(in_features=100, out_features=100, bias=True)
  (activation_2): ReLU()
  (linear_3): Linear(in_features=100, out_features=50, bias=True)
  (activation_3): ReLU()
  (linear_out): Linear(in_features=50, out_features=3, bias=True)
)
Input dim: 92 | Output dim: 3 | Trainable params: 34,703


In [11]:
mlp.train_model(optimizer='adam', loss='cross_entropy',
                regularization_type='L2', lambda_reg=1e-3,
                weights_data=True,
                epochs=100, batch_size=10, lr=2e-4, device='cpu')

Starting MLP training...
Epoch 001: train=6.2111e-01 | val=4.4007e+00
Epoch 010: train=3.1373e-01 | val=4.4397e+00
Epoch 020: train=9.7410e-02 | val=2.3857e-01
Epoch 030: train=1.0915e-01 | val=1.5469e+00
Epoch 040: train=5.7282e-02 | val=1.1337e+00
Epoch 050: train=1.1041e-01 | val=6.8859e-02
Epoch 060: train=5.5478e-02 | val=2.4298e-01
Epoch 070: train=2.3596e-02 | val=7.4670e-02
Epoch 080: train=3.2684e-02 | val=1.1864e-01
Epoch 090: train=2.6954e-02 | val=1.3077e-01
Epoch 100: train=1.8892e-02 | val=8.3244e-02
